# **README**

### 필수 과제  
해당 코드는 OpenAI의 API를 활용하여 GPT-4o-mini 버전의 챗봇을 호출하고, 특수한 분야(주식과 세금)의 문서를 PDF 형식으로 로드하여 청킹한 후, RAG 기법을 활용해 해당 문서에 특화된 user의 질문/요청에 대해 응답하는 챗봇을 생성하는 코드입니다.

**목차**  
1. 사용환경 준비 (API 입력)
2. OpenAI 모델 로드
3. 주식/세금 관련 PDF 문서 로드
4. 문서 내용 청킹
5. RAG 체인을 활용하기 위한 벡터 임베딩 생성
6. FAISS를 활용하여 벡터 스토어 생성
7. Retriever 변환
8. 프롬프트 템플릿 정의
9. RAG Chain 구성
10. 챗봇 구동/성능 확인 및 차이점 리뷰

### 도전 과제  
외부 경로의 Prompt 폴더에 있는 프롬프트 텍스트 파일(.txt)을 로드하여 gpt 모델에 입력한 후, 생성된 결과를 Result 폴데에 아래 형식으로 저장하는 코드입니다.
```
(파일명)_result_(타임스탬프).txt
```

In [2]:
# 1. 사용환경 준비
# 모델에 따른 사용 환경을 준비합니다. (OpenAI 선택)
# API 키값은 이전에 Slack 으로 지급받은 조별 키값을 사용했습니다. (8조)

import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass("OpenAI API key 입력: ")

In [3]:
# 2. 모델 로드하기
# OpenAI 모델을 로드하여 gpt-4o-min 모델 사용하는 model 객체를 생성합니다.

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini")

In [22]:
# 3. 필요한 데이터 불러오기 (문서 로드하기)
# 터미널에 pip install langchain pypdf 을 입력하여 관련 라이브러리를 설치합니다.

from langchain.document_loaders import PyPDFLoader

# PDF 파일 경로 지정
file_path = "2024_Stock_Tax_Book.pdf"  # 로컬 PDF 파일 경로

# PyPDFLoader를 사용해 문서 로드
loader = PyPDFLoader(file_path)

# 문서를 읽어 Document 객체로 변환
documents = loader.load()

# 결과 확인
for page, content in enumerate(documents[:4]):
    print(f"Page {page + 1}:")
    print(content.page_content)
    print("-" * 50)

Page 1:

--------------------------------------------------
Page 2:

--------------------------------------------------
Page 3:
지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5
국세청 자산과세국장
머리말
--------------------------------------------------
Page 4:
본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한
것으로 모든 법령규정을 담고 있지는 않습니다.
또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생
할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인
하시기 바랍니다.
본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  
유예된 법령 등은 반영되어 있지 않습니다.
본 책자에 표기된 세율은 지방소득세를 

In [21]:
# 4-1. 가져온 문서를 청킹하기 (I. CharacterTextSplitter 방식 사용)

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="|f",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(documents)

# 상위 10개의 청크 출력
print("Top 10 chunks:")
for i, chunk in enumerate(splits[:10], 1):
    print(f"\nChunk {i}:\n{chunk.page_content}")

Top 10 chunks:

Chunk 1:
지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5
국세청 자산과세국장
머리말

Chunk 2:
본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한
것으로 모든 법령규정을 담고 있지는 않습니다.
또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생
할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인
하시기 바랍니다.
본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  
유예된 법령 등은 반영되어 있지 않습니다.
본 책자에 표기된 세율은 지방소득세를 포함하지 않은 세율이며, 법령의 표기방식은  
일부 아래와 같이 간소화하여 표기하였습니다.
본 자료는 주식을 거래할 때 과세되는 세금에 대하여 납세자의 개략적인 이해를 돕기 
위해 일반적인 사항 위주로 수록하였으며, 개별사안에 대해 세법을 적용하는 경우 관련 
법령에 대한

In [5]:
# 4-2. 가져온 문서를 청킹하기 (II. RecursiveCharacterTextSplitter 방식 사용)

from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(documents)

# 상위 10개의 청크 출력
print("Top 10 chunks:")
for i, chunk in enumerate(splits[:10], 1):
    print(f"\nChunk {i}:\n{chunk.page_content}")

Top 10 chunks:

Chunk 1:
지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.

Chunk 2:
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서

Chunk 3:
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무

Chunk 4:
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득

Chunk 5:
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주

Chunk 6:
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5

Chunk 7:
2024.  5
국세청 자산과세국장
머리말

Chunk 8:
본 책자에 수록된 내용은 세법에서 규정하고 있는 내용을 알기 쉽게 요약하여 서술한
것으로 모든 법령규정을 담고 있지는 않습니다.

Chunk 9:
또한, 법령이 개정되거나 법령의 해석이 변경되어 본 책자의 내용과 다른 경우가 발생
할 수 있으므로 실제 사안에 적용하는 경우 반드시 관련 법령과 해석 등을 충분히 확인

Chunk 10:
하시기 바랍니다.
본 책자는 발간일 현재 개정된 법령 등을 기준으로 작성되었습니다. 다만, 시행이  
유예된 법령 등은 반영되어 있지 않습니다.


## CharacterTextSplitter

**동작 방식**  
지정한 **구분자(separator)**를 기준으로 텍스트를 분리합니다.  
각 청크의 길이가 chunk_size를 넘지 않도록 분리하며, 겹치는 부분(chunk_overlap)을 포함할 수 있습니다.
구분자 단위로만 동작하기 때문에 단순하고 빠릅니다.

**장점**
 - 단순함 : 구분자에 따라 분리하므로 이해하기 쉽고 빠르게 동작합니다.
 - 빠른 성능 : 재귀적인 처리를 하지 않기 때문에 속도가 빠릅니다.
사용자 지정 구분자 : 특정 구분자(예: 문단, 페이지)를 기준으로 분리하려는 경우 적합합니다.

**단점**
 - 구분자 의존 : 구분자가 없는 텍스트에서는 원하는 크기로 분리하지 못할 수 있습니다.
 - 유연성 부족 : 구분자를 넘어선 더 세밀한 분리가 어렵습니다. 예를 들어, 구분자가 없는 경우에도 적절히 분리해야 한다면 한계가 있습니다.

## RecursiveCharacterTextSplitter
**동작 방식**  
텍스트를 재귀적으로 분리합니다.  
긴 텍스트를 먼저 큰 단위에서(chunk_size) 나누고, 나머지 부분은 다시 작은 단위로 세부 분리합니다.  
기본적으로 문단 → 문장 → 단어 순으로 분리하며, 유연하게 처리할 수 있습니다.  

**장점**
 - 유연성 : 특정 구분자가 없더라도 적절한 크기로 분리되도록 재귀적으로 처리합니다. 또한, 문단, 문장, 단어 등의 계층 구조를 활용할 수 있어 문맥 보존에 유리합니다.
 - 문맥 보존 : 텍스트의 문맥을 고려하며, 가능한 한 의미 단위(문장 또는 문단)로 나누기 때문에 더 자연스러운 청킹 결과를 얻을 수 있습니다.
 - 텍스트 처리 : 긴 텍스트에서도 효율적으로 작동하며, 구분자가 없거나 일정하지 않은 경우에 적합합니다.

**단점**
 - 느린 속도 : 재귀적으로 처리하므로, 큰 텍스트의 경우 CharacterTextSplitter에 비해 속도가 느릴 수 있습니다.
 - 복잡성 : 내부적으로 복잡한 로직을 사용하므로 설정이나 디버깅이 어렵게 느껴질 수 있습니다.

### 파라미터 해석

 - separator="|f" : 청킹에 필요한 구분자를 정의합니다. 이 경우, PDF 파일의 페이지 단위로 분리하기 위해 |f 구분자를 사용했습니다.
 - chunk_size=100 : 하나의 청크에 포함될 최대 텍스트 길이를 지정합니다. 텍스트가 chunk_size보다 길 경우, 추가로 나뉘어지고, 단위는 length_function에 의해 결정됩니다.
 - chunk_overlap=10 : 인접한 청크 사이에서 겹치는 부분의 길이를 지정합니다. 해당 인자를 통해 청크 사이에 이어지는 문맥의 흐름이 소실되지 않도록 보완하는 역할을 합니다.
 - length_function=len : 텍스트의 길이를 계산하는 함수로, 기본값은 Python의 len() 함수를 사용하며, 텍스트의 문자 수를 기준으로 길이를 측정합니다.
 - is_separator_regex=False : 구분자를 정규 표현식으로 처리할지 여부를 결정합니다. False 일 경우, 단순 문자열로 간주합니다.

---

### 결론
해당 문서는 구분자로 명확하게 판단할 수 있는 페이지 단위로 문서가 구분되어 있어서 구분자 사용에 적합한 **CharacterTextSplitter** 청킹 방식을 채택했습니다.

In [6]:
# 5. Vector Embedding 생성

from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [7]:
# 6. Vector Store 생성
# 5에서 생성한 벡터 임베딩과 4에서 청킹한 데이터를 바탕으로 벡터스토어를 생성합니다.

import faiss
from langchain_community.vectorstores import FAISS


vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

In [8]:
# 7. FAISS를 Retriever로 변환
# 벡터 임베딩으로 저장된 vectorstore라는 FAISS 객체로부터 as_retriever 메서드를 활용해 retriever 인터페이스로 변환합니다.
# 이 메서드를 통해 vectorstore가 사용자로부터 질문 또는 쿼리를 받아 유사한 벡터를 검색할 수 있도록 설정합니다.
# search_kwargs 의 k값이 1로 설정되어 있으므로, 가장 유사한 벡터 1개를 retriever 에 반환합니다.

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

In [9]:
# 8. 프롬프트 템플릿 정의
# langchain_core 모듈에서 제공하는 ChatPromptTemplate 을 활용해 Chat 모델에서 사용할 프롬프트 템플릿을 정의합니다.
from langchain_core.prompts import ChatPromptTemplate

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

In [10]:
# 9. RAG 체인 구성
# 데이터가 RAG 체인과 같은 복잡한 파이프라인에서 일관성을 갖고 원활하게 전달되도록 RunnablePassthrough 를 불러옵니다. 이후, ContextToText 클래스에서 이를 상속받아 사용합니다.
# 또한, 데이터가 각 체인 단계를 거치며 올바르게 전달되고 있는지 확인하기 위한 목적으로 DebugPassThrough 클래스에서도 RunnablePassthrough 를 상속받아 사용합니다.
from langchain_core.runnables import RunnablePassthrough

# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,                    # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()        # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText()|   contextual_prompt | model


In [ ]:
# 10. 챗봇 구동/성능 확인
# 사용자가 "exit" 을 입력할 경우, 대화가 종료됩니다.
while True : 
	print("========================")
	query = input("질문을 입력하세요: ")
	if query == "exit" :
		break
	response = rag_chain_debug.invoke(query)
	print("Final Response:")
	print(response.content)

Debug Output: "시간외대량매매"에 대해 설명해줘.
Debug Output: {'context': [Document(metadata={'source': '2024_Stock_Tax_Book.pdf', 'page': 34}, page_content='시간외단일가매매의 주요내용은 다음과 같습니다.\n3) 시간외대량매매\n시간외대량매매는 시간외시장의 매매거래 시간 동안 매매수량단위의 5,000배(ETF·ETN')], 'question': '"시간외대량매매"에 대해 설명해줘.'}
Final Response:
시간외대량매매는 시간외시장에서 매매거래가 이루어지는 동안, 매매수량 단위의 5,000배에 해당하는 대량의 거래를 의미합니다.
Debug Output: "신주인수권증서"와 "신주인수권증권" 의 차이에 대해 설명해줘.
Debug Output: {'context': [Document(metadata={'source': '2024_Stock_Tax_Book.pdf', 'page': 41}, page_content='하며 거래가 이루어진다는 점에서 주식등 개념에 포함되며, 신주인수권 \n증서와 신주인수권증권으로 구분됩니다.\n정부가 금전차입을 목적으로 공공자금관리기금법에 의하여 국회의')], 'question': '"신주인수권증서"와 "신주인수권증권" 의 차이에 대해 설명해줘.'}
Final Response:
"신주인수권증서"와 "신주인수권증권"은 모두 신주인수권과 관련된 금융 자산이지만, 그 형태와 기능에서 차이가 있습니다. 

신주인수권증서는 특정한 권리를 부여하는 증서로, 주식을 인수할 수 있는 권리를 나타냅니다. 이는 주주가 신규 발행된 주식을 우선적으로 인수할 수 있는 권리를 가리킵니다.

반면, 신주인수권증권은 이러한 신주인수권의 거래 및 전환을 가능하게 하는 증권으로, 시장에서 사고팔 수 있는 형태로 존재합니다. 즉, 신주인수권증권은 신주인수권증서의 금융적 거래 가능성을 높여주는 역할을 합니다.

결론적으로, 신주인수권증서는 권리의 증명서이고, 신주인

&nbsp;
아래는 위 3개의 동일한 질문을 RAG가 적용되지 않은 일반 LLM 모델(Claude)에 요청했을 때 생성된 응답입니다.

**Claude**
```py
User : "시간외대량매매"에 대해 설명해줘.

Assistant : 시간외대량매매는 한국 주식시장에서 정규장 거래시간(09:00~15:30) 이후에 이루어지는 대량 주식거래를 말합니다.
주요 특징은 다음과 같습니다:
거래시간
장종료 후 15:40~16:00 (시간외 단일가 매매)
16:00~18:00 (시간외 대량매매)

거래단위
보통 최소 거래금액이 1억원 이상이어야 함
대부분 기관투자자나 외국인투자자들이 주로 이용

거래방식
매도자와 매수자가 직접 거래 상대방을 정하여 매매
매매가격은 당일 종가를 기준으로 함
호가 단위 제한없이 협의 가능

장점
대량매매로 인한 시장 충격을 최소화할 수 있음
정규장 종가 기준으로 거래되어 가격 안정성 확보
기관투자자들의 포트폴리오 조정에 유용

이러한 시간외대량매매는 기관투자자들의 포트폴리오 리밸런싱이나 대규모 자금 운용에 효과적인 거래 방식으로 활용되고 있습니다.
```
&nbsp;
```py
User : "신주인수권증서"와 "신주인수권증권" 의 차이에 대해 설명해줘.

Assistant : "신주인수권증서"와 "신주인수권증권"의 주요 차이점을 설명해드리겠습니다:

신주인수권증서 (Rights Certificate)
1. 발행 대상
- 기존 주주에게만 발행
- 주주의 신주인수권을 확인하는 증서

2. 발행 목적
- 기존 주주의 지분율 희석 방지
- 우선적 신주 청약권 보장

3. 특징
- 존속기간이 짧음 (보통 2주 이내)
- 거래 가능하나 기간이 제한적
- 별도 대가 없이 무상으로 발행

신주인수권증권 (Rights Warrant)
1. 발행 대상
- 일반 투자자 대상 발행 가능
- 기존 주주에 한정되지 않음

2. 발행 목적
- 기업의 자금 조달
- 투자자에게 신주 매입 기회 제공

3. 특징
- 존속기간이 비교적 김 (수개월~수년)
- 자유롭게 양도 가능
- 발행시 대가를 지불해야 함
- 주식으로 전환할 수 있는 권리만을 부여

주요 차이점 요약:
1. 발행대상의 차이: 기존 주주 vs 일반 투자자
2. 존속기간의 차이: 단기 vs 장기
3. 발행비용의 차이: 무상 vs 유상
4. 거래기간의 차이: 제한적 vs 자유로운 양도
```
&nbsp;
```py
User : 증권예탁증권(DR) 이 상장거래되는 거래소 5곳을 알려줘.


Assistant : 주요 DR(예탁증권) 상장거래소 5곳은 다음과 같습니다:

뉴욕증권거래소(NYSE)
세계 최대 규모의 DR 시장
ADR(American Depositary Receipt) 형태로 거래
엄격한 상장요건과 높은 유동성 특징


런던증권거래소(LSE)
GDR(Global Depositary Receipt) 중심
유럽 최대의 DR 거래시장
국제거래소(International Order Book) 운영


룩셈부르크증권거래소(LuxSE)
유럽의 주요 GDR 거래 중심지
특히 신흥국 기업들의 DR 상장 선호


싱가포르증권거래소(SGX)
아시아 지역의 주요 DR 거래소
TDR(Taiwan Depositary Receipt) 등 아시아 기업 DR 다수 상장


홍콩증권거래소(HKEX)
중국 기업 중심의 DR 거래
HDR(Hong Kong Depositary Receipt) 형태로 거래
중국 본토 기업들의 해외 자금조달 창구 역할
```
&nbsp;
RAG 기법을 통해 문서가 추가적으로 학습된 AI챗봇과 일반 Claude 의 답변 결과를 확인해보면, 첫 번째 질문의 시간외대량매매에 관한 설명에서 Claude 와 다르게 "매매수량 단위의 5,000배에 해당하는 대량의 거래" 라는 구체적인 정보를 언급한 차이가 있습니다. 두 번째 질문인 "신주인수권증서"와 "신주인수권증권"의 차이점에 관한 질문에서 Claude 는 다소 사전적 정의에 가까운 답변이 생성된 것에 비해, RAG가 적용된 AI 에서는 두 개념에 대해 좀 더 이해도가 높은 답변을 생성했으며 마지막에는 신주인수권증권이 신주인수권증서의 금융적 거래 가능성을 높여주는 역할임을 설명하여 두 개념에 대해 더 깊이있는 해석이 담긴 설명을 생성했습니다. 세 번째 질문인 증권예탁증권(DR) 이 상장거래되는 거래소 5곳에 관한 질문에서도 Claude에서 제시한 예시와 다른 결과를 생성했습니다.

생성형 AI 챗봇에서 RAG가 필요한 이유는 결국, 사용자의 질문이나 요청에 대해 보다 정확하고 관련성 높은 답변을 생성하기 위해서입니다.
RAG가 적용되기 전의 AI 모델들은 기본적으로 "사전에 학습"된 데이터에 한해서만 user의 요청을 받아들여 답변을 생성한다는 한계가 있었는데, RAG 체인을 활용하면 사전학습된 정보 이외에 특수한 분야의 전문적인 정보나 최신 동향이 포함된 정보들을 AI가 실시간으로 검색하고 이를 활용하기 때문에, 특정 분야에 전문화된 질문이나 다양한 요청에 대해 더 정확한 응답을 생성해낼 수 있습니다.

In [20]:
# 도전 과제 (외부에서 불러온 Prompt를 읽고, 결과값을 result.txt 형태로 저장하기)
import openai
import time

# 1. 프롬프트 디렉토리 및 결과 디렉토리 경로 설정
prompt_dir = "Prompts"
result_dir = "Results"

# 2. 프롬프트 파일 읽고 결과 파일 저장
for prompt_filename in os.listdir(prompt_dir):
    if prompt_filename.endswith(".txt"):
        # 프롬프트 파일 경로
        prompt_file_path = os.path.join(prompt_dir, prompt_filename)
        
        # 프롬프트 파일 읽기
        with open(prompt_file_path, "r", encoding="utf-8") as file:
            prompt = file.read()
        
        # 3. ChatGPT API 호출
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt},
            ]
        )
        
        # 4. 응답 추출
        generated_response = response.choices[0].message.content
        
        # 5. 타임스탬프 생성
        timestamp = int(time.time())
        
        # 6. 결과 파일 경로 생성
        result_filename = f"{os.path.splitext(prompt_filename)[0]}_result_{timestamp}.txt"
        result_file_path = os.path.join(result_dir, result_filename)
        
        # 7. 결과 파일에 저장
        with open(result_file_path, "w", encoding="utf-8") as result_file:
            result_file.write(f"Prompt:\n{prompt}\n\n")
            result_file.write(f"Response:\n{generated_response}\n")

        print(f"Result saved to {result_file_path}")


Result saved to Results\prompt1_result_1731933152.txt
Result saved to Results\prompt2_result_1731933159.txt
Result saved to Results\prompt3_result_1731933174.txt
